#### Import modules

In [1]:
cd ../../../../Apps/Python/bolsao-api

C:\Users\luisr\Desktop\Repositories\Apps\Python\bolsao-api


In [2]:
import requests, json, pandas as pd
from modules.stations import alertario_api_live, alertario_live, inmet_live, alertario_features, inmet_features

# from modules.points_monitoring import PointsMonitoring

C:\Users\luisr\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
import pandas as pd
from datetime import datetime
import pytz; tz_br = pytz.timezone('Brazil/East')
from scipy.spatial import distance_matrix

'To-Do:'
'1. Include timestamp on sorted_data function response'
'2. Change how polygons class handles non-point data in both status and general status'

class PointsMonitoring:

    def __init__(self, df1, df2, id1, id2, coords1, coords2, sep=' - '):
        self.df1=df1; self.df2=df2;
        self.id1=id1; self.id2=id2;
        self.coords1=coords1; self.coords2=coords2;
        self.distances = self.distance_matrix_df()
        self.distances_rank = self.distances_rank_df()
        self.sep = sep
        
    def distance_matrix_df(self):
        return pd.DataFrame(distance_matrix(self.df1[self.coords1], self.df2[self.coords2]), index=self.df1[self.id1], columns=self.df2[self.id2])

    def distances_rank_df(self):
        return pd.DataFrame(
            [row.sort_values().index.values for idx, row in self.distances.iterrows()],
            columns=range(1, self.distances.shape[1] + 1),
            index=self.distances.index,
        )

    def flat_features(self, data, id_col, features):
        return pd.concat([data.set_index(id_col).loc[_id, features].add_suffix(f' - {_id}') for _id in data[id_col].values], axis=0)

    def sorted_points_data(self, data, n_first, features, join_coords=True, join_props=False, as_datetime=False):
        cols = [f'{feat}{self.sep}{position}' for position in range(1, n_first + 1) for feat in features] 
        sorted_data = []; now = datetime.now(tz_br)
        for idx, row in self.distances_rank.iloc[:, :n_first].iterrows():
            sorted_data.append(self.flat_features(data.set_index(self.id2).loc[row.values].reset_index(), self.id2, features).values)
        sorted_data = pd.DataFrame(sorted_data, columns=cols, index=self.distances_rank.index)
        stamp = now if as_datetime else now.strftime('%Y-%m-%d %X')
        timestamp = pd.Series(stamp, index=sorted_data.index, name='timestamp')
        sorted_data = pd.concat([timestamp, sorted_data], axis=1).reset_index()
        if join_coords:
            sorted_data = self.df1[[self.id1]+self.coords1].join(sorted_data.set_index(self.id1), on=self.id1)
        if join_props:
            miss_cols = list(set(self.df1).difference(sorted_data).difference(self.coords1))
            sorted_data = self.df1[[self.id1] + miss_cols].join(sorted_data.set_index(self.id1), on=self.id1)
        return sorted_data.fillna('')

---

### Load stations data

In [137]:
alertario_stations = pd.read_csv('static/stations/alertario.csv')
inmet_stations = pd.read_csv('static/stations/inmet.csv')

working_alertario_stations = alertario_api_live()['id_estacao']
working_inmet_stations = inmet_live()['id_estacao']

alertario_stations['id_estacao'] = alertario_stations['id_estacao'].astype(str)
alertario_stations = alertario_stations[alertario_stations['id_estacao'].isin(working_alertario_stations)]
inmet_stations = inmet_stations[inmet_stations['id_estacao'].isin(working_inmet_stations)]

### Load flood polygons data

In [138]:
polygons = pd.read_csv('static/clusters/clusters_micro.csv').rename(columns={'sublabel': 'cluster_id'})
polygons = polygons[polygons['cluster_id'] != -1]

### Load cameras data

In [139]:
cameras = pd.read_csv('static/city/cameras.csv')

cameras.loc[660, 'Longitude'] = - cameras.loc[660, 'Longitude']

---

### polygons closest stations data

In [163]:
df1 = polygons
df2 = alertario_stations
id1 = 'cluster_id'
id2 = 'id_estacao'
coords1 = ['lng_centroid', 'lat_centroid']
coords2 = ['longitude', 'latitude']

data = alertario_api_live()
features = alertario_features
n_first = 5

poly_monitor = PointsMonitoring(df1, df2, id1, id2, coords1, coords2)

poly_stations = poly_monitor.sorted_points_data(data, features, n_first)

display(poly_monitor.distances_rank.head())
display(poly_stations.head())

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
cluster_id,,,,,,,,,,,,,,,,,,,,,
0,17,28,1,18,21,19,16,7,33,14,...,12,24,25,8,26,29,20,22,27,3
1,31,5,2,15,4,32,6,16,33,1,...,24,30,12,25,29,26,20,22,27,3
2,6,2,31,5,16,4,1,33,15,32,...,30,24,12,25,29,26,20,22,27,3
3,6,16,1,31,2,5,4,33,28,15,...,30,24,12,25,29,26,20,22,27,3
4,17,28,1,16,21,18,7,33,19,4,...,24,12,8,25,26,29,20,22,27,3


,cluster_id,lng_centroid,lat_centroid,timestamp,acumulado_chuva_15_min - 1,acumulado_chuva_1_h - 1,acumulado_chuva_4_h - 1,acumulado_chuva_24_h - 1,acumulado_chuva_96_h - 1,acumulado_chuva_15_min - 2,...,acumulado_chuva_15_min - 4,acumulado_chuva_1_h - 4,acumulado_chuva_4_h - 4,acumulado_chuva_24_h - 4,acumulado_chuva_96_h - 4,acumulado_chuva_15_min - 5,acumulado_chuva_1_h - 5,acumulado_chuva_4_h - 5,acumulado_chuva_24_h - 5,acumulado_chuva_96_h - 5
1,0,-43.310232,-23.006631,2023-02-07 02:51:12,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,2.0
2,1,-43.176842,-22.926423,2023-02-07 02:51:12,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,11.8,0.0,0.0,0.0,0.0,2.0
3,2,-43.185999,-22.966793,2023-02-07 02:51:12,0.0,0.0,0.0,0.0,2.4,0.0,...,0.0,0.0,0.0,0.0,4.2,0.0,0.0,0.0,0.0,5.0
4,3,-43.202540,-22.979735,2023-02-07 02:51:12,0.0,0.0,0.0,0.0,2.4,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8
5,4,-43.298339,-23.012736,2023-02-07 02:51:12,0.0,0.0,0.0,0.0,12.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2.0


### Cameras closest stations data

In [165]:
df1 = cameras
id1 = 'Codigo'
coords1 = ['Longitude', 'Latitude']
df2 = alertario_stations
id2 = 'id_estacao'
coords2 = ['longitude', 'latitude']

data = alertario_api_live()
features = alertario_features
n_first = 5

cams_monitor = PointsMonitoring(df1, df2, id1, id2, coords1, coords2)

cams_stations = cams_monitor.sorted_points_data(data, features, n_first)

display(cams_monitor.distances_rank.head())
display(cams_stations.head())

,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
Codigo,,,,,,,,,,,,,,,,,,,,,
1,15,5,31,32,4,2,33,16,6,8,...,24,30,12,25,29,26,20,22,27,3
2,15,5,31,32,4,2,33,16,6,8,...,24,30,12,25,29,26,20,22,27,3
3,15,5,32,31,4,2,33,16,7,6,...,24,30,12,25,29,26,20,22,27,3
4,8,9,23,11,32,13,15,10,7,33,...,19,12,30,29,26,25,20,22,27,3
5,15,5,31,2,32,4,33,6,16,7,...,24,30,12,25,29,26,20,22,27,3


,Codigo,Longitude,Latitude,timestamp,acumulado_chuva_15_min - 1,acumulado_chuva_1_h - 1,acumulado_chuva_4_h - 1,acumulado_chuva_24_h - 1,acumulado_chuva_96_h - 1,acumulado_chuva_15_min - 2,...,acumulado_chuva_15_min - 4,acumulado_chuva_1_h - 4,acumulado_chuva_4_h - 4,acumulado_chuva_24_h - 4,acumulado_chuva_96_h - 4,acumulado_chuva_15_min - 5,acumulado_chuva_1_h - 5,acumulado_chuva_4_h - 5,acumulado_chuva_24_h - 5,acumulado_chuva_96_h - 5
0,1,-43.177031,-22.900259,2023-02-07 02:51:18,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,2.0
1,2,-43.179391,-22.901392,2023-02-07 02:51:18,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,2.0
2,3,-43.190353,-22.904902,2023-02-07 02:51:18,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0
3,4,-43.251544,-22.809417,2023-02-07 02:51:18,0.0,0.0,0.0,0.6,21.0,0.0,...,0.0,0.0,0.0,0.2,3.8,0.0,0.0,0.0,0.0,2.4
4,5,-43.174155,-22.913741,2023-02-07 02:51:18,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,2.4


---
### Time measuring for city cameras

In [166]:
import time

df1 = cameras
id1 = 'Codigo'
coords1 = ['Longitude', 'Latitude']

start = time.time()
cams_stations = PointsMonitoring(df1, df2, id1, id2, coords1, coords2)
cams_sorted_alertario = cams_stations.sorted_points_data(data, features, n_first)
end = time.time()

print(f'n,k permutaions time (s): {end - start}')

display(cams_stations.distances_rank.head())
display(cams_sorted_alertario.head())

n,k permutaions time (s): 26.131662845611572


,1,2,3,4,5,6,7,8,9,10,...,24,25,26,27,28,29,30,31,32,33
Codigo,,,,,,,,,,,,,,,,,,,,,
1,15,5,31,32,4,2,33,16,6,8,...,24,30,12,25,29,26,20,22,27,3
2,15,5,31,32,4,2,33,16,6,8,...,24,30,12,25,29,26,20,22,27,3
3,15,5,32,31,4,2,33,16,7,6,...,24,30,12,25,29,26,20,22,27,3
4,8,9,23,11,32,13,15,10,7,33,...,19,12,30,29,26,25,20,22,27,3
5,15,5,31,2,32,4,33,6,16,7,...,24,30,12,25,29,26,20,22,27,3


,Codigo,Longitude,Latitude,timestamp,acumulado_chuva_15_min - 1,acumulado_chuva_1_h - 1,acumulado_chuva_4_h - 1,acumulado_chuva_24_h - 1,acumulado_chuva_96_h - 1,acumulado_chuva_15_min - 2,...,acumulado_chuva_15_min - 4,acumulado_chuva_1_h - 4,acumulado_chuva_4_h - 4,acumulado_chuva_24_h - 4,acumulado_chuva_96_h - 4,acumulado_chuva_15_min - 5,acumulado_chuva_1_h - 5,acumulado_chuva_4_h - 5,acumulado_chuva_24_h - 5,acumulado_chuva_96_h - 5
0,1,-43.177031,-22.900259,2023-02-07 02:51:49,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,2.0
1,2,-43.179391,-22.901392,2023-02-07 02:51:49,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.4,0.0,0.0,0.0,0.0,2.0
2,3,-43.190353,-22.904902,2023-02-07 02:51:49,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0
3,4,-43.251544,-22.809417,2023-02-07 02:51:49,0.0,0.0,0.0,0.6,21.0,0.0,...,0.0,0.0,0.0,0.2,3.8,0.0,0.0,0.0,0.0,2.4
4,5,-43.174155,-22.913741,2023-02-07 02:51:49,0.0,0.0,0.0,0.0,11.8,0.0,...,0.0,0.0,0.0,0.0,0.8,0.0,0.0,0.0,0.0,2.4
